In [1]:
import sys
import pickle
import numpy as np
from omegaconf import OmegaConf

sys.path.append('../')

In [2]:
LOG = False
ITERATION = 1
AUGMENT = False
DATA_KEY = 'prostate'
extra_description = 'moredata'

cfg = OmegaConf.load('../configs/conf.yaml')
OmegaConf.update(cfg, 'run.iteration', ITERATION)
OmegaConf.update(cfg, 'run.data_key', DATA_KEY)

unet_name = 'monai-64-4-4'
args = unet_name.split('-')
cfg.unet[DATA_KEY].pre = unet_name
cfg.unet[DATA_KEY].arch = args[0]
cfg.unet[DATA_KEY].n_filters_init = None if unet_name == 'swinunetr' else int(args[1])
cfg.wandb.project = f'{DATA_KEY}_{unet_name}_{ITERATION}{extra_description}'
cfg.wandb.log = LOG

if args[0] == 'monai':
    cfg.unet[DATA_KEY].depth = int(args[2])
    cfg.unet[DATA_KEY].num_res_units = int(args[3])


In [3]:
N_DIMS = [2, 4, 8, 16, 32, 64]

layer_names = [
    'model.1.submodule.1.submodule.1.submodule.0.conv.unit3.conv',
    'model.1.submodule.1.submodule.2.0.conv'
]
name = layer_names[0].replace('.', '_')
if AUGMENT: name += '_aug'

for n_dims in N_DIMS:
    print(f'Loading IPCA of {n_dims} dims')
    with open(f'/workspace/src/out/pca/{cfg.wandb.project}_{name}_{n_dims}dim.pkl',  'rb') as f:
        ipca = pickle.load(f)
    evr = ipca.explained_variance_ratio_
    cum_evr = np.cumsum(evr)
    # num_comp = np.argmax(cum_evr >= 0.95) + 1
    print(f'Max retained variance with all components: {cum_evr[-1]}')
    print(f'Loading activation maps obtained with IPCA of {n_dims} dims')
    act = np.load(f'/workspace/src/out/activations/{cfg.wandb.project}_{name}_activations_{n_dims}dims.npy')
    print(f'Shape of activations: {act.shape}')

Loading IPCA of 2 dims
Max retained variance with all components: 0.16919846756572138
Loading activation maps obtained with IPCA of 2 dims
Shape of activations: (986, 2)
Loading IPCA of 4 dims
Max retained variance with all components: 0.24581898489647303
Loading activation maps obtained with IPCA of 4 dims
Shape of activations: (986, 4)
Loading IPCA of 8 dims
Max retained variance with all components: 0.33115446005511306
Loading activation maps obtained with IPCA of 8 dims
Shape of activations: (986, 8)
Loading IPCA of 16 dims
Max retained variance with all components: 0.4159668188844243
Loading activation maps obtained with IPCA of 16 dims
Shape of activations: (986, 16)
Loading IPCA of 32 dims
Max retained variance with all components: 0.5098525702053119
Loading activation maps obtained with IPCA of 32 dims
Shape of activations: (986, 32)
Loading IPCA of 64 dims
Max retained variance with all components: 0.6223144601442684
Loading activation maps obtained with IPCA of 64 dims
Shape 